In [48]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np  
import json
import random
import re
import random


In [49]:
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCH = 150

### Data Preprocessing and Cleaning


In [50]:
# data cleaning and preprocessing method inspired by https://www.kaggle.com/code/jocelyndumlao/chatbot-for-mental-health-conversations
with open("./data/json/intents_final.json") as f:
    dataset = json.load(f)
    
df = pd.DataFrame(dataset["intents"])

dic = {"tag":[], "patterns":[], "responses":[]}
for i in range(len(df)):
    patterns = df[df.index == i]['patterns'].values[0]
    responses = df[df.index == i]['responses'].values[0]
    tags = df[df.index == i]['tag'].values[0]
    for j in range(len(patterns)):
        dic['tag'].append(tags)
        dic['patterns'].append(patterns[j])
        dic['responses'].append(responses)
        
df = pd.DataFrame.from_dict(dic)


In [51]:
# word tokenization for patterns, which is input patterns expected from users
tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df['patterns'])
tokenizer.get_config()

VOCAB_SIZE = len(tokenizer.word_index)
print('number of unique words = ', VOCAB_SIZE)

patterns2seq = tokenizer.texts_to_sequences(df['patterns'])
x = pad_sequences(patterns2seq, padding='post')
print('X shape = ', x.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['tag'])
UNIQUE_WORDS = len(np.unique(y))
print('y shape = ', y.shape)
print('num of classes = ', UNIQUE_WORDS)


number of unique words =  828
X shape =  (373, 143)
y shape =  (373,)
num of classes =  92


### 2-layer LSTM model


In [52]:
def create_model():
    model = Sequential()
    model.add(Input(shape = (x.shape[1],)))
    model.add(Embedding(input_dim = VOCAB_SIZE+1, output_dim = 100, mask_zero = True))
    model.add(LSTM(512, return_sequences = True))
    model.add(LayerNormalization())
    model.add(LSTM(512))
    model.add(LayerNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(UNIQUE_WORDS, activation='softmax'))
    return model


### Model Training


In [53]:
model = create_model()
model.compile(optimizer = tf.optimizers.Adam(learning_rate = LEARNING_RATE), 
                loss ='sparse_categorical_crossentropy', 
                metrics = ['accuracy'])
model_history = model.fit(x=x,
                          y=y,
                          batch_size=BATCH_SIZE,
                          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3, mode = "max")],
                          epochs=EPOCH)
model.save("./model/best_model.h5")

Epoch 1/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 603ms/step - accuracy: 0.0977 - loss: 4.8531
Epoch 2/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 572ms/step - accuracy: 0.5689 - loss: 1.8623
Epoch 3/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 605ms/step - accuracy: 0.8368 - loss: 0.5476
Epoch 4/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 607ms/step - accuracy: 0.9305 - loss: 0.2569
Epoch 5/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 820ms/step - accuracy: 0.9485 - loss: 0.1682
Epoch 6/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 719ms/step - accuracy: 0.9883 - loss: 0.0728
Epoch 7/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 554ms/step - accuracy: 0.9898 - loss: 0.0587
Epoch 8/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 583ms/step - accuracy: 0.9795 - loss: 0.0797
Epoch 9/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 617ms/step - accuracy: 0.9861 - loss: 0.0744


### Chatbot Testing


In [54]:
model = load_model("./model/best_model.h5")

In [55]:
def wally(query):
    text = []
    txt = re.sub('[^a-zA-Z\']', ' ', query)
    txt = txt.lower()
    txt = txt.split()
    txt = " ".join(txt)
    text.append(txt)
        
    x_test = tokenizer.texts_to_sequences(text)
    x_test = np.array(x_test).squeeze()
    try:
        x_test = pad_sequences([x_test], padding='post', maxlen=x.shape[1])
    except:
        x_test = pad_sequences([[x_test]], padding='post', maxlen=x.shape[1])
    y_pred = model.predict(x_test)
    y_pred = y_pred.argmax()
    tag = lbl_enc.inverse_transform([y_pred])[0]
    responses = df[df['tag'] == tag]['responses'].values[0]

    return random.choice(responses)
    
while True:
    user_input = input("User: ")
    response = wally(user_input)
    print("Wally: {}".format(response))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
Wally: Hi there. How are you feeling today?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Wally: Hello there. Glad to see you're back. What's going on in your world right now?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Wally: I'm Wally, a Therapeutic AI Assitant designed to assist you. Tell me about yourself.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Wally: I'm Wally, your Personal Therapeutic AI Assistant. How are you feeling today?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Wally: I exist thanks to the skills and vision of @type_nak.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Wally: How long have you been feeling this way?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Wally: Many people find peer support a helpful tool that can aid in their recovery. There are a variety of organizations that offer support groups for consumers, their family members and friends. Some support groups are peer-led, while others may be led by a mental health professional.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44